In [ ]:
import polars as pl
import pandas as pd
import numpy as np
import networkx as nx
import os
from rtsvg import *
rt = RACETrack()
_base_network_ = '0'
_base_dir_     = '../../../data/stanford/facebook/'
_layout_file_  = _base_dir_ + _base_network_ + '.layout.parquet'
_edges_ = open(_base_dir_ + _base_network_ + '.edges', 'rt').read()
_lu_ = {'fm':[], 'to':[]}
for _edge_ in _edges_.split('\n'):
    if _edge_ == '': continue
    _lu_['fm'].append(_edge_.split(' ')[0])
    _lu_['to'].append(_edge_.split(' ')[1])
df = pl.DataFrame(_lu_)
relates = [('fm', 'to')]
g   = rt.createNetworkXGraph(df, relates)
pos = nx.spring_layout(g)
ln_params = {'relationships':relates, 'pos':pos, 'draw_labels':False, 'node_size':'small'}
_rtg_ = rt.interactiveGraphPanel(df, ln_params, w=800, h=600)
if os.path.exists(_layout_file_): _rtg_.loadLayout(_layout_file_)
len(df), len(set(df['fm'])), len(set(df['to'])), len(set(df['fm']) | set(df['to']))
_link_ = rt.link(df, relates, pos, node_size='small', w=400, h=300)
_link_.renderSVG()
import hdbscan
node_ls = list(pos.keys())
node_xy = []
for node in node_ls:
    node_xy.append(pos[node])
node_xy = np.array(node_xy)
cluster = hdbscan.HDBSCAN(min_cluster_size=10).fit(node_xy)
lu_xy = {'x':[], 'y':[], 'node':[], 'cluster':[]} 
for i in range(len(node_ls)):
    lu_xy['x'].append(node_xy[i][0]), lu_xy['y'].append(node_xy[i][1])
    lu_xy['node'].append(node_ls[i]), lu_xy['cluster'].append(cluster.labels_[i])
_xy_hdbscan_= rt.xy(pd.DataFrame(lu_xy), x_field='x', y_field='y', color_by='cluster', w=400, h=300)
rt.tile([_link_, _xy_hdbscan_])

In [ ]:
_rtg_